In [2]:
!pip install mittens


In [3]:
import tqdm
import tensorflow as tf
import numpy as np
import tqdm
import pickle
import numpy
import os
import csv
from mittens import Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from scipy import spatial 


2022-07-23 22:14:57.968757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-23 22:14:57.968783: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aaron/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
%run constants.py
tf.random.set_seed(
    SEED
)
np.random.seed(SEED)

In [5]:
with open(X_TRAIN_INPUT_SAVE_FILE_PRE_VEC, "rb") as f:
        x_train = pickle.load(f)
with open(VOCAB_SAVE_FILE, "rb") as f:
        vocab = pickle.load(f)

In [6]:
def get_embedding_index():
    embeddings_index = {}

    f = open(GLOVE_300D_FILEPATH)
    for line in tqdm.tqdm(f, ncols=100, desc="Loading Glove Embeddings."):
        values = line.split()
        word = values[0]
        coefs = numpy.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()

    return embeddings_index

In [7]:
ei = get_embedding_index()

Loading Glove Embeddings.: 400000it [00:15, 25602.81it/s]


In [ ]:
train_corpora = x_train[29].str.split()
train_corpora = train_corpora.tolist()
flat_list_train_corpora = [x for xs in train_corpora for x in xs]
flat_list_train_corpora = list(set(flat_list_train_corpora))
flat_list_train_corpora

a = [flat_list_train_corpora.lower() for x in flat_list_train_corpora]
a

In [ ]:

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed


pre_glove = glove2dict(GLOVE_300D_FILEPATH)
sw = list(_stop_words.ENGLISH_STOP_WORDS)
train_corpora_for_glove = [token.lower() for token in flat_list_train_corpora if (token.lower() not in sw)]
oov = [token for token in train_corpora_for_glove if token not in pre_glove.keys()]

corp_vocab = list(set(oov))
brown_doc = [' '.join(train_corpora_for_glove)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=EMBEDDING_DIM, max_iter=10000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))


In [ ]:
new_glove_merged = newglove | ei
len(new_glove_merged)

In [ ]:
try:
    os.remove(FINE_TUNED_GLOVE_300D_FILEPATH)
except OSError:
    pass
with open(FINE_TUNED_GLOVE_300D_FILEPATH, "w") as file:
    for k, v in new_glove_merged.items():
        dimensions = ""
        for x in v:
            dimensions += str(x) + " "
        glove_line = k + " " + dimensions
        file.write(glove_line)

In [ ]:

def find_closest_embeddings(embedding_index, embedding): 
    return sorted(embedding_index.keys(), key=lambda word: spatial.distance.euclidean(embedding_index[word], embedding))

In [ ]:
find_closest_embeddings(new_glove_merged, new_glove_merged['libstdc'])[0:10]

In [ ]:
#Should fail - as libstdc is NOT in the original glove matrix
find_closest_embeddings(ei, ei['libstdc'])[0:10]